In [1]:
import pandas as pd
import numpy as np

# Set display options
pd.options.display.precision = 2

Load the data set

In [32]:
customer = pd.read_csv("Customer Acqusition.csv")
spend = pd.read_csv('spend.csv', parse_dates=['Month'])
repayment = pd.read_csv('Repayment.csv', parse_dates=['Month'])

### QUESTION #1 

In case age is less than 18, replace it with mean of age values.

In [3]:
df_cust = customer.copy() 

df_cust['Age'] = np.where(df_cust.Age < 18, df_cust.Age.mean(), df_cust.Age)
df_cust.sample(5)

,No,Customer,Age,City,Product,Limit,Company,Segment
55,56,A56,60.0,CHENNAI,Platimum,500000.0,C18,Normal Salary
34,35,A35,55.0,COCHIN,Platimum,100001.0,C17,Govt
47,48,A48,42.0,TRIVANDRUM,Platimum,10001.0,C10,Self Employed
8,9,A9,79.0,CHENNAI,Silver,100003.0,C9,Govt
11,12,A12,51.0,BOMBAY,Gold,500000.0,C12,Self Employed


### QUESTION #2 

In case spend amount is more than the limit, replace it with 50% of that customer’s limit.

In [4]:
df_cust_spend = pd.merge(left=df_cust,
               right=spend,
               on='Customer',
               how='inner')

df_cust_spend['Amount'] = np.where(df_cust_spend.Amount > df_cust_spend.Limit, 0.50 * df_cust_spend.Limit, df_cust_spend.Amount)
df_cust_spend.sample(5)

,No,Customer,Age,City,Product,Limit,Company,Segment,Sl No:,Month,Type,Amount
772,37,A37,61.0,BANGALORE,Platimum,100003.0,C19,Normal Salary,657,2004-01-15,SANDALS,50001.50
860,39,A39,78.0,TRIVANDRUM,Platimum,500000.0,C21,Salaried_MNC,1398,2006-03-31,FOOD,416672.65
434,22,A22,27.0,BOMBAY,Silver,100000.0,C22,Self Employed,508,2005-02-23,SHOPPING,50000.00
1334,58,A58,32.0,CALCUTTA,Silver,100002.0,C20,Normal Salary,1417,2004-01-15,AIR TICKET,50001.00
1306,57,A57,37.0,TRIVANDRUM,Silver,100001.0,C19,Normal Salary,1155,2006-11-03,PETRO,13756.30


### QUESTION #3 

Incase the repayment amount is more than the limit, replace the repayment with the limit. 

In [34]:
df_cust_repay = pd.merge(left = df_cust,
               right = repayment,
               left_on = 'Customer',
               right_on = 'Customer',
               how = 'inner')

df_cust_repay['Amount'] = np.where(df_cust_repay.Amount > df_cust_repay.Limit, df_cust_repay.Limit, df_cust_repay.Amount)
df_cust_repay.sample(5)

,No,Customer,Age,City,Product,Limit,Company,Segment,SL No:,Month,Amount
719,34,A34,34.0,CALCUTTA,Platimum,100000.0,C16,Salaried_Pvt,828,2005-02-16,45750.90
707,33,A33,19.0,BOMBAY,Platimum,10002.0,C15,Salaried_MNC,885,2006-05-03,10002.00
1245,54,A54,46.0,COCHIN,Platimum,500000.0,C16,Normal Salary,54,2005-12-03,215719.14
589,28,A28,36.0,TRIVANDRUM,Gold,500000.0,C10,Salaried_MNC,381,2006-07-11,319405.61
74,6,A6,70.0,DELHI,Silver,100000.0,C6,Self Employed,192,2004-01-03,100000.00


### QUESTION #4

a. How many distinct customers exist?

In [35]:
customer.Customer.nunique()

100

b. How many customers based on their segment who buy gold products?

In [38]:
pd.crosstab(
    index=df_cust_spend.Product,
    columns=df_cust_spend.Segment,
)

Segment,Govt,Normal Salary,Salaried_MNC,Salaried_Pvt,Self Employed
Product,,,,,
Gold,96,149,92,129,144
Platimum,96,235,76,94,69
Silver,82,57,76,46,59


c. What is the average monthly spend by customers?

In [40]:
pd.pivot_table(
    data=df_cust_spend,
    index=['Customer'],
    columns=df_cust_spend.Month.dt.month,
    values='Amount',
    aggfunc='mean')\
.loc[['A1', 'A2', 'A3']].round(3)

Month,1,2,3,4,5,6,7,8,9,10,11,12
Customer,,,,,,,,,,,,
A1,372762.59,175235.69,NaN,261649.25,361141.31,NaN,NaN,344372.1,NaN,244534.68,183839.61,NaN
A2,50000.00,50000.00,NaN,50000.00,50000.00,NaN,NaN,NaN,50000.0,NaN,62864.18,NaN
A3,5000.00,5000.00,5000.0,5000.00,5000.00,NaN,NaN,NaN,NaN,5000.00,NaN,5000.0


d. What is the average monthly repayment by customers?

In [9]:
pd.pivot_table(
    data=df_cust_repay,
    index='Customer',
    columns=df_cust_repay.Month.dt.month,
    values='Amount',
    aggfunc='mean')\
.loc[['A1', 'A2', 'A3']]

Month,1,2,3,4,5,6,7,8,9,10,11,12
Customer,,,,,,,,,,,,
A1,500000.0,500000.0,NaN,500000.0,NaN,NaN,500000.0,NaN,500000.0,NaN,500000.0,NaN
A2,100000.0,100000.0,NaN,100000.0,NaN,NaN,NaN,100000.0,NaN,100000.0,100000.0,NaN
A3,10000.0,10000.0,NaN,10000.0,10000.0,NaN,NaN,NaN,10000.0,NaN,10000.0,NaN


e. If the monthly rate of interest is 2.9%, what is the profit for the bank for each month?

In [10]:
def funct(value):
    if value > 0:
        return value * 0.029
    else:
        return 0 

In [44]:
df_profit = df_cust_spend.copy()[['Month', 'Amount']]

df_profit['Profit'] = df_cust_repay.Amount - df_profit.Amount

df_profit.Profit = df_profit.Profit.apply(funct)

df_profit.groupby([df_profit.Month.dt.month]).agg({'Profit': 'sum'})

,Profit
Month,
1,558292.36
2,486756.16
3,357319.59
4,247621.26
5,351343.14
6,80955.46
7,43624.03
8,57713.25
9,73458.97


f. What are the top 5 product types?

In [49]:
df_cust_spend.Type.value_counts()

PETRO           200
FOOD            160
CAMERA          160
AIR TICKET      147
TRAIN TICKET    132
SHOPPING        113
BUS TICKET       99
JEWELLERY        95
CLOTHES          95
MOVIE TICKET     76
RENTAL           76
BIKE             49
AUTO             40
CAR              30
SANDALS          28
Name: Type, dtype: int64

In [48]:
df_cust_spend.groupby('Type').agg({'Amount': 'sum'}).nlargest(5, columns='Amount')

,Amount
Type,
PETRO,2.86e+07
CAMERA,2.77e+07
FOOD,2.05e+07
AIR TICKET,2.02e+07
TRAIN TICKET,2.00e+07


g. Which city is having maximum spend?

In [50]:
df_cust_spend.groupby('City').agg({'Amount': 'sum'}).nlargest(1, columns='Amount')

,Amount
City,
COCHIN,4.60e+07


h. Which age group is spending more money?

In [53]:
df_cust_spend.groupby('Age').agg({'Amount': 'sum'}).nlargest(1, columns='Amount')

,Amount
Age,
28.0,1.74e+07


i. Who are the top 10 customers in terms of repayment?

In [54]:
df_cust_repay.groupby('Customer').agg({'Amount': 'sum'}).nlargest(10, columns='Amount')

,Amount
Customer,
A61,1.05e+07
A60,9.88e+06
A13,9.57e+06
A43,8.49e+06
A45,8.45e+06
A12,8.33e+06
A14,7.94e+06
A44,7.74e+06
A39,7.62e+06


### QUESTION #5 

Calculate the city wise spend on each product on yearly basis.

In [16]:
pd.pivot_table(
    data=df_cust_spend, 
    index=['City', 'Product'],
    columns=df_cust_spend.Month.dt.year,
    values='Amount',
    aggfunc='sum'
)

Month                    2004      2005      2006
City       Product                               
BANGALORE  Gold      9.29e+06  1.29e+07  1.20e+07
           Platimum  1.11e+06  9.12e+05  6.86e+05
           Silver    1.29e+06  8.31e+05  1.42e+06
BOMBAY     Gold      6.99e+06  8.98e+06  7.92e+06
           Platimum  8.97e+05  7.12e+05  9.72e+05
           Silver    5.32e+05  5.09e+05  1.09e+06
CALCUTTA   Gold      7.48e+06  1.03e+07  1.12e+07
           Platimum  2.04e+06  4.43e+06  3.59e+06
           Silver    5.00e+05  4.40e+05  1.16e+06
CHENNAI    Gold      1.06e+06  3.74e+06  2.70e+06
           Platimum  3.54e+06  5.05e+06  5.02e+06
           Silver    2.82e+05  3.77e+05  3.50e+05
COCHIN     Gold      7.32e+06  1.21e+07  1.05e+07
           Platimum  3.42e+06  4.96e+06  4.92e+06
           Silver    8.14e+05  6.52e+05  1.27e+06
DELHI      Gold      2.81e+06  1.96e+06  2.96e+06
           Platimum  2.84e+05  4.94e+05  1.77e+05
           Silver    7.68e+05  6.12e+05  8.55e+05
PATNA      Gold      2.07e+06  3.25e+06  2.28e+06
           Platimum  1.11e+06  2.88e+06  1.26e+06
           Silver    7.57e+05  9.50e+05  7.00e+05
TRIVANDRUM Gold      2.42e+06  1.24e+06  2.74e+06
           Platimum  2.16e+06  3.60e+06  2.15e+06
           Silver    2.10e+05  2.70e+05  5.56e+05

### QUESTION #6

Show monthly comparison of total spends, city wise

In [60]:
pd.pivot_table(
    data=df_cust_spend, 
    index='City',
    columns=df_cust_spend.Month.dt.month,
    values="Amount",
    aggfunc='sum'
)

Month,1,2,3,4,5,6,7,8,9,10,11,12
City,,,,,,,,,,,,
BANGALORE,9.04e+06,6.73e+06,5.36e+06,4.21e+06,6.38e+06,1.33e+06,2.23e+06,6.95e+05,7.03e+05,1.09e+06,2.26e+06,422167.24
BOMBAY,5.95e+06,4.28e+06,5.23e+06,1.87e+06,3.89e+06,1.17e+06,1.78e+05,1.07e+06,1.28e+06,6.76e+05,2.87e+06,140040.71
CALCUTTA,9.06e+06,8.15e+06,4.94e+06,3.46e+06,5.58e+06,2.29e+06,2.23e+06,1.55e+06,1.51e+06,1.96e+05,2.08e+06,55001.50
CHENNAI,4.47e+06,4.39e+06,3.18e+06,2.52e+06,2.86e+06,1.24e+06,6.94e+05,3.06e+05,7.46e+05,1.56e+05,1.35e+06,223628.27
COCHIN,1.17e+07,5.75e+06,5.65e+06,3.88e+06,6.58e+06,2.50e+06,1.29e+06,2.32e+06,1.38e+06,8.69e+05,3.13e+06,909533.05
DELHI,2.71e+06,1.49e+06,2.27e+06,7.77e+05,1.60e+06,5.11e+05,5.00e+04,1.55e+05,5.00e+04,6.46e+04,3.92e+05,842342.02
PATNA,4.92e+06,2.35e+06,1.44e+06,2.34e+06,2.14e+06,7.46e+05,5.00e+04,4.19e+05,7.03e+04,3.30e+05,4.14e+05,50000.00
TRIVANDRUM,3.70e+06,2.86e+06,2.39e+06,1.20e+06,2.81e+06,4.00e+05,2.60e+05,4.54e+05,1.86e+05,2.86e+05,1.43e+05,640922.26


### QUESTION #7 

Show comparison of yearly spend on air tickets

In [18]:
df_cust_spend[df_cust_spend.Type == 'AIR TICKET'].groupby(df_cust_spend.Month.dt.year).agg({'Amount': 'sum'})

,Amount
Month,
2004,4.36e+06
2005,1.06e+07
2006,5.25e+06


### QUESTION #8

Show comparison of monthly spend for each product

In [61]:
pd.pivot_table(
    data=df_cust_spend, 
    index='Product',
    columns=df_cust_spend.Month.dt.month,
    values='Amount',
    aggfunc='sum'
)

Month,1,2,3,4,5,6,7,8,9,10,11,12
Product,,,,,,,,,,,,
Gold,3.24e+07,2.48e+07,1.93e+07,1.32e+07,2.15e+07,6.54e+06,5.17e+06,4.95e+06,3.82e+06,3.02e+06,9.20e+06,2.33e+06
Platimum,1.50e+07,9.13e+06,7.29e+06,5.18e+06,8.15e+06,2.97e+06,1.29e+06,1.71e+06,1.67e+06,5.50e+05,2.74e+06,7.50e+05
Silver,4.21e+06,2.09e+06,3.91e+06,1.90e+06,2.14e+06,6.90e+05,5.30e+05,2.93e+05,4.43e+05,1.00e+05,6.91e+05,2.01e+05


### QUESTION #9

Write user defined function to perform the following analysis:

In [22]:
def top_customers():
    product = input("Enter product name : ").capitalize().strip()
    time = int(input("Enter year  : "))

    mask = (df_cust_repay.Product == product) & (df_cust_repay.Month.dt.year == time)
    
    top10 = df_cust_repay[mask].groupby(['City', 'Customer']).agg({'Amount': 'sum'}).reset_index()

    return top10.groupby(['City']).apply(lambda x: x.nlargest(10, 'Amount'))

top_customers()
            

City Customer    Amount
City                                        
BANGALORE  2    BANGALORE      A14  4.50e+06
           4    BANGALORE      A43  4.00e+06
           0    BANGALORE       A1  3.50e+06
           1    BANGALORE      A13  3.00e+06
           3    BANGALORE      A30  2.00e+06
           5    BANGALORE      A63  1.00e+06
BOMBAY     8       BOMBAY      A42  6.00e+06
           6       BOMBAY      A12  4.00e+06
           7       BOMBAY      A17  3.00e+06
           9       BOMBAY      A62  3.00e+06
CALCUTTA   14    CALCUTTA      A60  5.00e+06
           10    CALCUTTA      A10  2.00e+06
           11    CALCUTTA      A15  2.00e+06
           12    CALCUTTA      A29  2.00e+06
           13    CALCUTTA      A31  5.00e+05
CHENNAI    15     CHENNAI      A27  3.50e+06
           16     CHENNAI      A67  5.00e+05
COCHIN     20      COCHIN      A45  5.50e+06
           21      COCHIN      A61  5.50e+06
           18      COCHIN      A16  2.50e+06
           17      COCHIN      A11  1.50e+06
           19      COCHIN      A32  5.00e+05
           22      COCHIN      A65  5.00e+05
           23      COCHIN      A94  5.00e+05
DELHI      24       DELHI      A44  4.50e+06
           25       DELHI      A93  5.00e+05
PATNA      26       PATNA      A26  4.00e+06
           27       PATNA      A66  5.00e+05
           28       PATNA      A95  5.00e+05
TRIVANDRUM 29  TRIVANDRUM      A28  3.00e+06